In [8]:
import numpy as np
np.set_printoptions(precision=2)

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler, Normalizer

import seaborn

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
%matplotlib inline

from sklearn.datasets import load_digits
digits = load_digits()
#print(digits.DESCR)

X = np.sqrt(digits.data)
Y = digits.target

X_train,X_test,Y_train, Y_test = tts(X,Y,train_size=0.8,test_size=0.2,random_state=0)

import pandas as pd
#pd.set_eng_float_format(accuracy=1) #, use_eng_prefix=True)
pd_X_train = pd.DataFrame(X_train)
c = pd_X_train.corr()
print(c.iloc[25:36,25:36])


plt.imshow(c.values,cmap='gray_r')

X_train.mean(axis=0)

fig, ax = plt.subplots(1,2,figsize=(20,12))
# Original image
seaborn.heatmap(X_train[5].reshape((8,8)), cmap='gray_r',ax=ax[0], square=True, cbar=False,
            annot=True, fmt='0.1f', vmin=-5, vmax=5)
ax[0].set_title('Original (sqrt) image')

X_tr_mean = X_train.mean(axis=0)
X_tr_std  = X_train.std(axis=0)
#print(X_std.reshape((8,8)))
#print(X_tr_std.reshape ((8,8))[:,0:9])
print(X_tr_mean.reshape((8,8))[:,0:4],'\n',
X_tr_std.reshape((8,8))[:,0:4])


fig, ax = plt.subplots(1,2,figsize=(20,12))
# Original image
seaborn.heatmap(X_tr_std.reshape((8,8)), ax=ax[0], square=True, cbar=False,
            annot=True, fmt='0.1f', vmin=-5, vmax=5,cmap='gray_r')
seaborn.heatmap(X_tr_mean.reshape((8,8)), cmap='gray_r',ax=ax[1], square=True, cbar=False,
            annot=True, fmt='0.1f', vmin=-5, vmax=5)
ax[0].set_title('Std image')
ax[1].set_title('Mean image')

In [9]:
def normalize(x_train,x_test): # to get a probability out of a non-negative vector
    sc = Normalizer('l1').fit(x_train)
    X_train_normalized = sc.transform(x_train)
    X_test_normalized = sc.transform(x_test)
    return [X_train_normalized, X_test_normalized]

In [10]:
std_scale = StandardScaler() # for mean and std

std_scale.fit(X_train)
print(std_scale.scale_.reshape((8,8)))
# we see that the scaler does not divide by zero which is reasonable
# is PCA dropping the deterministic attributes?

X_train_std = std_scale.transform(X_train)
X_test_std  = std_scale.transform(X_test)

pca = PCA(n_components=10)
pca.fit(X_train_std)
X_train_pca, X_test_pca = pca.transform(X_train_std), pca.transform(X_test_std)

print(pca.components_[0])
x = pca.components_
x.shape
print(x[:,[0,32,39]]) # indeed PCA drops the dead attributes

def up_distance(x1,x2):
    y1, y2 = pca.transform(x1),pca.transform(x2) # should I scale it first?
    y12 = y1 - y2 
    return np.linalg.norm(y12)

up_distance(np.ones((1,64)),np.zeros((1,64)))
tmp = pca.transform(np.ones((1,64)))
print(tmp)
np.linalg.norm(tmp)

scaling does not commute with pca
https://stats.stackexchange.com/questions/69157/why-do-we-need-to-normalize-data-before-principal-component-analysis-pca